<a href="https://colab.research.google.com/github/jincy-p-janardhanan/SPP-Pneumonia-Net/blob/ml/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

In [ ]:
normal = 0
bacterial = 0
viral = 0

# Dataset: [COVID-19 Radiography Database](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database)

## Download dataset

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/radiography/"

In [ ]:
% cd /content/drive/MyDrive/radiography/
% pwd

In [ ]:
! kaggle datasets download -d tawsifurrahman/covid19-radiography-database

In [ ]:
!unzip \*.zip && rm *.zip

## Copy Images

Copy normal images to main_project/normal

In [ ]:
normal_folder = '/content/drive/MyDrive/radiography/COVID-19_Radiography_Dataset/Normal'

In [ ]:
# change working directory before copying files
% cd '/content/drive/MyDrive/radiography/COVID-19_Radiography_Dataset/Normal'
! pwd

In [ ]:
files = os.listdir(normal_folder)
for f in files:
  if 'Normal' in f and normal < 2780:
    normal += 1
    shutil.copy(normal_folder+f,'/content/drive/MyDrive/main_project/Normal')
print("normal: ", normal, "\t bacterial: ", bacterial, "\t viral: ", viral)

Copy viral pneumonia images to main_project/Viral

In [ ]:
viral_folder = '/content/drive/MyDrive/radiography/COVID-19_Radiography_Dataset/Viral Pneumonia'

In [ ]:
% cd '/content/drive/MyDrive/radiography/COVID-19_Radiography_Dataset/Viral Pneumonia'
! pwd

In [ ]:
files = os.listdir(viral_folder)
for f in files:
  if 'Viral' in f and viral < 2780:
    viral += 1
    shutil.copy(pneumonia_folder+f,'/content/drive/MyDrive/main_project/Viral')

# Dataset: [Chest X-ray Images Pneumonia](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia)

## Download dataset

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/paultimothymooney/"

In [ ]:
% cd /content/drive/MyDrive/paultimothymooney/
% pwd

In [ ]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!unzip \*.zip && rm *.zip

## Copy Images

Copy bacterial and viral pneumonia images from train folder to main_project/Bacterial and main_project/Viral

In [ ]:
pneumonia_folder = '/content/drive/MyDrive/paultimothymooney/chest_xray/train/PNEUMONIA/'

In [ ]:
% cd /content/drive/MyDrive/paultimothymooney/chest_xray/train/PNEUMONIA/
! pwd

In [ ]:
for f in files:
  if 'bacteria' in f and bacterial < 2780:
    bacterial += 1
    shutil.copy(pneumonia_folder+f,'/content/drive/MyDrive/main_project/Bacterial')
  elif 'virus' in f and viral < 2780:
    viral += 1
    shutil.copy(pneumonia_folder+f,'/content/drive/MyDrive/main_project/Viral')
print("normal: ", normal, "\t bacterial: ", bacterial, "\t viral: ", viral)

Copy the 8 bacterial pneumonia images in val folder to main_project/Bacterial

In [ ]:
pneumonia_folder = '/content/drive/MyDrive/paultimothymooney/chest_xray/val/PNEUMONIA/'

In [ ]:
% cd /content/drive/MyDrive/paultimothymooney/chest_xray/val/PNEUMONIA/
! pwd

In [ ]:
for f in files:
  if 'bacteria' in f and bacterial < 2780:
    bacterial += 1
    shutil.copy(pneumonia_folder+f,'/content/drive/MyDrive/main_project/Bacterial')
print("normal: ", normal, "\t bacterial: ", bacterial, "\t viral: ", viral)